In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prétraitement

In [ ]:
import os

# Chemin d'accès au dossier contenant les images
path = "/content/drive/MyDrive/Expériementation_Adience_DB/train_data"

# Compter le nombre d'images
count = 0
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".jpg") or (".png"):
            count += 1

# Afficher le résultat
print(f"Le dossier '{path}' contient {count} images.")


Le dossier '/content/drive/MyDrive/Expériementation_Adience_DB/train_data' contient 15651 images.


In [ ]:
import os
import numpy as np
from PIL import Image

# chemin du dossier contenant les images
folder_path = '/content/drive/MyDrive/Expériementation_Adience_DB/train_data'

# Liste des noms de tous les sous-dossiers
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

# Créer une liste vide pour stocker les tableaux d'images
X_train = []
# Créer une liste vide pour stocker les étiquettes
X_labels = []

# Parcourir chaque sous-dossier
for i, subfolder in enumerate(subfolders):
    # Liste des fichiers dans le sous-dossier
    file_names = [f.path for f in os.scandir(subfolder) if f.is_file()]
    # Parcourir chaque fichier
    for file_name in file_names:
        # Ouvrir l'image avec PIL
        image = Image.open(file_name)
        image = image.convert('RGB')
        # Redimensionner l'image à (299, 299)
        image = image.resize((64,64))
        # Convertir l'image en tableau numpy
        image_array = np.array(image)
        # Ajouter le tableau d'image à la liste
        X_train.append(image_array)
        # Ajouter l'étiquette à la liste
        X_labels.append(i)

X_train = np.array(X_train)


In [ ]:
print(len(X_train))
print(len(X_labels))

15651
15651


In [ ]:


import os
import numpy as np
from PIL import Image

# chemin du dossier contenant les images
folder_path = '/content/drive/MyDrive/Expériementation_Adience_DB/test_data'

# Liste des noms de tous les sous-dossiers
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

# Créer une liste vide pour stocker les tableaux d'images
Y_test = []
# Créer une liste vide pour stocker les étiquettes
Y_labels= []

# Parcourir chaque sous-dossier
for i, subfolder in enumerate(subfolders):
    # Liste des fichiers dans le sous-dossier
    file_names = [f.path for f in os.scandir(subfolder) if f.is_file()]
    # Parcourir chaque fichier
    for file_name in file_names:
        # Ouvrir l'image avec PIL
        image = Image.open(file_name)
        image = image.convert('RGB')
        # Redimensionner l'image à (299, 299)
        image = image.resize((64,64))
        # Convertir l'image en tableau numpy
        image_array = np.array(image)
        # Ajouter le tableau d'image à la liste
        Y_test.append(image_array)
        # Ajouter l'étiquette à la liste
        Y_labels.append(i)


Y_test = np.array(Y_test)


In [ ]:
print(len(Y_labels))
print(len(Y_test))

3716
3716


In [ ]:
print(len(Y_labels))
print(len(Y_test))
print(len(X_train))
print(len(X_labels))

3716
3716
15651
15651


# VGG19


In [ ]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import vgg19
from keras.layers import Dropout,Dense,Conv2D,GlobalAveragePooling2D
#from keras.preprocessing import image
num_classes=2284
IMAGE_SIZE = [64,64]  

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg19 = vgg19.VGG19(input_shape = IMAGE_SIZE + [3],weights='imagenet',include_top=False)

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg19.layers:
    layer.trainable = False
x = Flatten()(vgg19.output)
#x1 = Dense(512, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
output_layer = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg19.input, outputs = output_layer)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [ ]:
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
X_train = np.array(X_train)
X_labels = np.array(X_labels)
Y_test = np.array(Y_test)
Y_labels = np.array(Y_labels)



In [ ]:
model.fit(X_train, X_labels, batch_size=100, epochs=1000, validation_data=(Y_test,Y_labels))

Epoch 1/1000
157/157 [==============================] - 22s 72ms/step - loss: 32.3814 - accuracy: 0.0100 - val_loss: 24.9419 - val_accuracy: 0.0237
Epoch 2/1000
157/157 [==============================] - 9s 60ms/step - loss: 20.0172 - accuracy: 0.0496 - val_loss: 19.7738 - val_accuracy: 0.0562
Epoch 3/1000
157/157 [==============================] - 9s 60ms/step - loss: 13.4980 - accuracy: 0.1268 - val_loss: 16.7703 - val_accuracy: 0.0912
Epoch 4/1000
157/157 [==============================] - 10s 66ms/step - loss: 9.1168 - accuracy: 0.2376 - val_loss: 14.9007 - val_accuracy: 0.1267
Epoch 5/1000
157/157 [==============================] - 10s 67ms/step - loss: 6.1471 - accuracy: 0.3604 - val_loss: 13.7038 - val_accuracy: 0.1577
Epoch 6/1000
157/157 [==============================] - 10s 62ms/step - loss: 4.1861 - accuracy: 0.4928 - val_loss: 12.9105 - val_accuracy: 0.1878
Epoch 7/1000
157/157 [==============================] - 11s 68ms/step - loss: 2.8898 - accuracy: 0.5987 - val_loss: 1

# Resultat VGG19

In [ ]:
model.evaluate(Y_test, Y_labels, verbose=2)

117/117 - 7s - loss: 5.0034 - accuracy: 0.5630 - 7s/epoch - 61ms/step


[5.003390789031982, 0.5629709362983704]

In [ ]:
model.save('/content/drive/MyDrive/Adience/VGG19+SVM_Adience_DB.h5')

# SVM

In [ ]:
pip install keract


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow import keras
model=keras.models.load_model('/content/drive/MyDrive/Adience/VGG19+SVM_Adience_DB.h5')

OSError: ignored

In [ ]:
from keract import get_activations
model.summary()

In [ ]:
activations1 = get_activations(model,X_train[:1000],layer_names='flatten')
activations2 = get_activations(model,X_train[1000:2000],layer_names='flatten')
activations3 = get_activations(model,X_train[2000:3000],layer_names='flatten')
activations4 = get_activations(model,X_train[3000:4000],layer_names='flatten')
activations5 = get_activations(model,X_train[4000:5000],layer_names='flatten')
activations6 = get_activations(model,X_train[5000:6000],layer_names='flatten')
activations7 = get_activations(model,X_train[6000:7000],layer_names='flatten')
activations8 = get_activations(model,X_train[7000:8000],layer_names='flatten')
activations9 = get_activations(model,X_train[8000:9000],layer_names='flatten')
activations10 = get_activations(model,X_train[9000:10000],layer_names='flatten')
activations11= get_activations(model,X_train[10000:11000],layer_names='flatten')
activations12= get_activations(model,X_train[11000:12000],layer_names='flatten')
activations13= get_activations(model,X_train[12000:13000],layer_names='flatten')
activations14= get_activations(model,X_train[13000:14000],layer_names='flatten')
activations15= get_activations(model,X_train[14000:15000],layer_names='flatten')
activations16= get_activations(model,X_train[15000:],layer_names='flatten')

In [ ]:
X_train1= activations1
X_train1= X_train1["flatten"]
X_train2= activations2
X_train2= X_train2["flatten"]
X_train3= activations3
X_train3= X_train3["flatten"]
X_train4= activations4
X_train4= X_train4["flatten"]
X_train5= activations5
X_train5= X_train5["flatten"]
X_train6= activations6
X_train6= X_train6["flatten"]
X_train7= activations7
X_train7= X_train7["flatten"]
X_train8= activations8
X_train8= X_train8["flatten"]
X_train9= activations9
X_train9= X_train9["flatten"]
X_train10= activations10
X_train10= X_train10["flatten"]
X_train11= activations11
X_train11= X_train11["flatten"]
X_train12= activations12
X_train12= X_train12["flatten"]
X_train13= activations13
X_train13= X_train13["flatten"]
X_train14= activations14
X_train14= X_train14["flatten"]
X_train15= activations15
X_train15= X_train15["flatten"]
X_train16= activations16
X_train16= X_train16["flatten"]


In [ ]:
X_train =np.concatenate((X_train1,X_train2,X_train3,X_train4,X_train5,X_train6,X_train7,X_train8,X_train9,X_train10,X_train11,X_train12,X_train13,X_train14,X_train15,X_train16), axis=0)


In [ ]:
X_train.shape

(15651, 2048)

In [ ]:
activations17 = get_activations(model,Y_test[:1000],layer_names='flatten')
activations18 = get_activations(model,Y_test[1000:2000],layer_names='flatten')
activations19 = get_activations(model,Y_test[2000:3000],layer_names='flatten')
activations20 = get_activations(model,Y_test[3000:],layer_names='flatten')

In [ ]:
Y_test1= activations17
Y_test1= Y_test1['flatten']
Y_test2= activations18
Y_test2= Y_test2['flatten']
Y_test3= activations19
Y_test3= Y_test3['flatten']
Y_test4= activations20
Y_test4= Y_test4['flatten']


In [ ]:
Y_test =np.concatenate((Y_test1,Y_test2,Y_test3,Y_test4), axis=0)


In [ ]:
import pandas as pd
from sklearn.svm import SVC

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
model1 = SVC(kernel='rbf',probability=True, C=100)

In [ ]:
model1.fit(X_train,X_labels)

SVC(C=100, probability=True)

In [ ]:
predictions1 = model1.predict(X_train)

In [ ]:
predictions2 = model1.predict(Y_test)

In [ ]:
percentage = model1.score(Y_test,Y_labels)

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
score_train = accuracy_score(X_labels, predictions1)
score_test = accuracy_score(Y_labels, predictions2)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=99.987, test=46.878


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
res = confusion_matrix(Y_labels, predictions2)
for i in range(0,len(res)):
  print(res[i])
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(Y_test)}")
print(f"Accuracy = {score_test*100} %")
print(classification_report(Y_labels, predictions2))

[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 1 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.